# 生成 AI による記事アイデアの作成

この Notebook では、生成 AI に記事のネタだしから調査、記事作成までを自動で実行する Agent を作成します。

PV 数の多い記事を分析して得た PV 数が高くなりそうな記事のネタの仮説を元に、Web 検索を利用し情報を集めて記事アイデアを提案するという設計にします。

## Agent の作成

まずは、 `langchain` を使用した ReACT エージェントを作成します。

In [2]:
!pip install "langchain==0.0.309" langchainhub -qU
!pip install "duckduckgo-search>=3.9.5" -qU

In [13]:
import langchain
from langchain.llms.bedrock import Bedrock
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools.render import render_text_description
from langchain.agents import Tool, AgentExecutor
from langchain.agents.output_parsers import ReActSingleInputOutputParser
# from langchain.agents.format_scratchpad import format_log_to_str
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain import hub
import boto3

langchain.verbose = True

# Initialize Model
bedrock_client = boto3.client("bedrock-runtime")
llm = Bedrock(
    model_id="anthropic.claude-instant-v1",
    client=bedrock_client,
    model_kwargs={ 'max_tokens_to_sample': 1024 }
)

# Initialize Tool
wrapper = DuckDuckGoSearchAPIWrapper(region="jp-jp", safesearch="strict")
search = DuckDuckGoSearchRun(api_wrapper=wrapper)
tools = [
    Tool(
        name="duckduckgo-search",
        func=search.run,
        description="日本語もしくは英語でウェブ検索が可能",
    )
]

デフォルトの `langchain` の Agent のプロンプトは GPT 用に作成されているため、`Claude` の[プロンプトベストプラクティス](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)を反映したプロンプトでテンプレートを上書きします。

In [14]:
import re
from typing import Union

from langchain.agents.agent import AgentOutputParser
from langchain.agents.mrkl.prompt import FORMAT_INSTRUCTIONS
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from typing import List, Tuple

FINAL_ANSWER_ACTION = "<Final Answer>"
MISSING_ACTION_AFTER_THOUGHT_ERROR_MESSAGE = (
    "Invalid Format: Missing 'Action' after 'Thought"
)
MISSING_ACTION_INPUT_AFTER_ACTION_ERROR_MESSAGE = (
    "Invalid Format: Missing 'Action Input' after 'Action'"
)
FINAL_ANSWER_AND_PARSABLE_ACTION_ERROR_MESSAGE = (
    "Parsing LLM output produced both a final answer and a parse-able action:"
)


class ClaudeReActSingleInputOutputParser(AgentOutputParser):

    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        includes_answer = FINAL_ANSWER_ACTION in text
        regex = (
            r"<Action>[\s]*(.*?)[\s]*</Action>[\s]*<Action Input>[\s]*(.*)[\s]*</Action Input>"
        )
        action_match = re.search(regex, text, re.DOTALL)
        if action_match:
            if includes_answer:
                raise OutputParserException(
                    f"{FINAL_ANSWER_AND_PARSABLE_ACTION_ERROR_MESSAGE}: {text}"
                )
            action = action_match.group(1).strip()
            action_input = action_match.group(2)
            tool_input = action_input.strip(" ")
            tool_input = tool_input.strip('"')

            return AgentAction(action, tool_input, text)

        elif includes_answer:
            return AgentFinish(
                {"output": text.split(FINAL_ANSWER_ACTION)[-1].replace("</Final Answer>", "").strip()}, text
            )

        if not re.search(r"<Action>[\s]*(.*?)[\s]*</Action>", text, re.DOTALL):
            raise OutputParserException(
                f"Could not parse LLM output: `{text}`",
                observation=MISSING_ACTION_AFTER_THOUGHT_ERROR_MESSAGE,
                llm_output=text,
                send_to_llm=True,
            )
        elif not re.search(
            r"[\s]<Action Input>[\s]*(.*)[\s]*</Action Input>", text, re.DOTALL
        ):
            raise OutputParserException(
                f"Could not parse LLM output: `{text}`",
                observation=MISSING_ACTION_INPUT_AFTER_ACTION_ERROR_MESSAGE,
                llm_output=text,
                send_to_llm=True,
            )
        else:
            raise OutputParserException(f"Could not parse LLM output: `{text}`")

    @property
    def _type(self) -> str:
        return "react-single-input"

    
def format_log_to_str(
    intermediate_steps: List[Tuple[AgentAction, str]],
    observation_prefix: str = "<Observation>",
    observation_suffix: str = "</Observation>",
    llm_prefix: str = "<Thought>",
) -> str:
    """Construct the scratchpad that lets the agent continue its thought process."""
    thoughts = ""
    for action, observation in intermediate_steps:
        thoughts += action.log
        thoughts += f"\n{observation_prefix}{observation}{observation_suffix}\n{llm_prefix}"
    return thoughts

In [26]:
# Define Prompt
prompt = hub.pull("hwchase17/react")
tool_names = ", ".join([t.name for t in tools])
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=tool_names,
)
print("元のプロンプト: ", prompt.template)
print("---")
prompt.template = """
Human: 以下の instruction で与えられた指示に従ってください。以下のツールを利用することが可能です。

<ツール>
{tools}
</ツール>

以下のフォーマットを使用してください:

<output-format>
<Thought>指示に答えるために何をするべきか検討</Thought>
<Action>実行するアクション。必ず {tool_names} から選択する。</Action>
<Action Input>Action への入力</Action Input>
<Observation>アクションの結果</Observation>
... (Thought/Action/Action Input/Observation を N 回繰り返す)
<Thought>答えるのに必要な情報が揃いました</Thought>
<Summary>調査の結果分かったことの要約</Summary>
<Final Answer><Title>タイトル</Title><Body>記事本文</Body></Final Answer>
</output-format>

それでは以下から開始してください。

<instruction>
Instruction: {input}
</instruction>

Assistant:
<Thought>{agent_scratchpad}
"""
print("Claude に最適化されたプロンプト: ", prompt.template)

# Stop Generation on stop token (passed to Bedrock)
llm_with_stop = llm.bind(stop=["\n<Observation>"])

# Create Agent
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps'])
} | prompt | llm_with_stop | ClaudeReActSingleInputOutputParser()

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

元のプロンプト:  Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
---
Claude に最適化されたプロンプト:  
Human: 以下の instruction で与えられた指示に従ってください。以下のツールを利用することが可能です。

<ツール>
{tools}
</ツール>

以下のフォーマットを使用してください:

<output-format>
<Thought>指示に答えるために何をするべきか検討</Thought>
<Action>実行するアクション。必ず {tool_names} から選択する。</Action>
<Action Input>Action への入力</Action Input>
<Observation>アクションの結果</Observation>
... (Thought/Action/Action Input/Observation を N 回繰り返す)
<Thought>答えるのに必要な情報が揃いました</Thought>
<Summa

## 出力のハイライト

著作権の配慮および Hallucination の検知のために、出力のうち、検索で取得したデータをハイライトして表示します。

In [27]:
from IPython.display import display, HTML

colors = [
    "#FF0000",
    "#00FF00",
    "#FFFF00",
    "#FF00FF",
    "#00FFFF",
    "#CC0000",
    "#00CC00",
    "#CCCC00",
    "#CC00CC",
    "#00CCCC",
    "#990000",
    "#009900",
    "#999900",
    "#990099",
    "#009999",
]


def create_ngram(string, n=2):
    delimiter = ['「', '」', '…', '　']
    if n == 1:
        return list(string)
    elif n == 2:
        double = list(zip(string[:-1], string[1:]))
        double = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter))), double)
        return list(double)
    elif n == 3:
        triple = list(zip(string[:-2], string[1:-1], string[2:]))
        triple = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter) or (x[2] in delimiter))), triple)
        return list(triple)
    return []


def create_ngrams(string):
    return create_ngram(string, 1)


def highlight(string, source):
    return f'<span class="c{source}">{"".join(string)}</span>'


def process_result_japanese(result):
    output = result["output"].replace("<output>", "").replace("</output>", "")
    search_results = "".join([x[1].replace("...", "\n\n") for x in result["intermediate_steps"]]).split("\n\n")

    # 一文字ごとにトークンとして分割
    output_tokens = list(output)
    search_tokens = [list(search_result) for search_result in search_results]
    
    # ngram: 1
    ngram_dict = {}
    for idx, search_token in enumerate(search_tokens):
        ngrams = create_ngrams(search_token)
        for ngram in ngrams:
            if not ngram in ngram_dict:
                ngram_dict[ngram] = set()
            ngram_dict[ngram] = ngram_dict[ngram] | set([idx])
    
    # Match longest consecutive ngrams from single source and highlight
    html = ""
    buffer = ()
    buffer_source = set()
    used_source = set()
    for token in output_tokens:
        if not token in ngram_dict:
            if buffer:
                source = list(buffer_source)[0]
                html += highlight("".join(buffer), source)
                buffer = ()
                used_source |= set([source])
                buffer_source = set()
            html += token
        else:
            current_source = ngram_dict[token]
            if not buffer:
                buffer = [token]
                buffer_source = current_source
            else:
                # Check for overlap 
                if buffer_source & current_source:
                    buffer.append(token)
                    buffer_source = buffer_source & current_source
                # No overlap, flush buffer
                else:
                    source = list(buffer_source)[0]
                    html += highlight("".join(buffer), source)
                    buffer = [token]
                    used_source |= set([source])
                    buffer_source = current_source
    if buffer:
        html += highlight("".join(buffer), list(buffer_source)[0])
    
    html = html.replace("\n", "<br/>")
    style = "<style>" + "\n".join([
        f".c{source} " + "{ color: " + colors[idx % len(colors)] + " !important; }" for idx, source in enumerate(used_source)
    ]) + "</style>"

    return style + "<div>" + html + "</div>"

# processed_result = process_result(result)
# # print(processed_result)
# HTML(processed_result)

## Agent の検証

In [20]:
result = agent_executor.invoke({
    "input": "あなたは敏腕記者です。今話題の日本の芸能人で話題になりそうなものを一つピックアップし深掘りして関連情報などを調査し1500文字程度のニュース記事を作成してください。"
})
print("--- 出力 ---")
HTML(process_result_japanese(result))



> Entering new AgentExecutor chain...
指示に答えるために何をするべきか検討
</Thought>

<Action>
duckduckgo-search
</Action>

<Action Input>
今話題の日本の芸能人
</Action Input>
ニュース 特集 1～20／136,029 件 1 2 3 4 5 次へ 星野源が歌う『オードリーANN』東京ドーム主題歌ついに解禁! 若林&春日がしびれ、サトミツは涙 2日深夜放送のニッポン放送『オードリーのオールナイトニッポン（ANN）』（毎週土曜 深1：00）では、来年2月18... 2023-12-03 01:56... #松下奈緒 「セリフが覚えられない」神田正輝が3週連続で旅サラダ欠席…"激やせ"の裏で漏らしていた異変 2023/12/02 12:50 「神田さんは現在、体のメンテナンス中で、しばらくお休みをいただくことになりました」12月2日放送の『朝だ! 生です旅サラダ』（ABCテレビ・テレビ朝日系）に、またしても神田正輝（72）の姿はなかった。 欠席はこれで3週連続。 共演者の松下奈緒（38）はオープニングで冒頭のように語った。 11月18日の欠席では「72歳で初めて体のメンテナンスをすることになりました。 きちんとメンテナンスをしてすぐ戻ります」と #体調 #神田正輝 #情報番組 落合陽一氏が手がけた秋葉原のイルミネーション 昨年からの"激変"ぶりに一部で戸惑いの声 芸能ニュースランキング | ORICON NEWS 芸能 1～20／99 件 1 2 3 4 5 次へ 1 横田真悠、超ミニスカで太もも全開"美脚コーデ"を紹介「どの日の衣装が好き？ 」 モデルで俳優の横田真悠（24）が30日、自身のインスタグラムを更新。 美太もも輝く超ミニスカルックなど、写真で撮影衣装を紹介した。 2023-11-30 13:57 ｜SNS発｜ 2... ヤー! が候補入りした、なかやまきんに君 (C)ORICON NewS inc. 2位は上半期にはランクインなかった 【なかやまきんに君】 が急浮上。 今年1月には昨年12月31日付をもって、吉本興業とのマネジメント契約を終了したことを発表。 その理由に、海外進出を志し、お笑いよりもフィットネスの仕事の比重が高くなった

## 記事アイデアの作成

Agent がインターネットから必要な情報を検索して記事を書く検証ができたため、次は PV 数が高い記事を分析した結果をもとに、ヒットする記事の仮説を立てて必要な情報を収集し記事の草案を作成させます。

In [28]:
import pandas as pd
import os

output_path = "analysis.csv"
if os.path.exists(output_path):
    hits = pd.read_csv(output_path)
    for idx, row in hits.iterrows():
        print(row["analysis"])
        print("")

- クイズ形式の記事は読者の知識を試す面白さがある
- テレビ番組に関するクイズは、多くの人が親しみを感じる題材
- ジェパディの実際の問題を使っていることで、クイズの難易度と面白さが保証される
- 記事タイトルがクイズ形式で、読者の興味を引く
- 記事の概要で、クイズの形式と内容が明確に説明されている
- 同じ形式と題材のクイズ記事を定期的に提供することで、読者を獲得できる

- タイトルに「最も」「全米No.1」といった強調表現を用い、読者の興味を引く
- 高級ホテルという非日常的な体験に対する人々の憧れをテーマにする
- セレブリティが訪れるホテルという著名人との接点をアピールする
- 掲載ホテルの写真を多用し、非日常的な雰囲気を伝える 
- SNSでのシェア数をアピールすることで話題性を高める

- 「あらゆるタイプの人」「あらゆる予算」というキーワードがあるので、多くの人に関心のある普遍的なテーマだと思われる。
- 「驚くべき」「完璧な」といった言葉でクリック率を高めている。
- 誕生日やクリスマスなどの贈り物選びのシーズンに合わせたタイミングの良い記事。
- Reviewedという信頼できる情報源からの贈り物のおすすめ記事。
- 具体的な商品のおすすめがあるため、実際に購入に結びつきやすい。

- セレブリティの金銭トラブルは注目を集める話題だ。セレブの破産や巨額の報酬、資産など金銭に関するゴシップ記事を作ることでPVを獲得できる。
- 過去10年間の出来事をまとめたリスト形式は読みやすくPVを集めやすい。時代背景を踏まえた10年間のトレンドを分析した記事を作れば再現できる。
- 著名人のスキャンダルやゴシップは関心が高く、クリック率が良い。セレブの金銭問題を扱った記事を作ればPVを獲得できる。

- 有名人の軍歴に関する記事であるため、有名人の過去の秘話や意外な一面に関心がある読者を引きつけている
- 具体的な有名人の名前が記載されているため、その有名人のファンも記事に興味を持つ
- 軍隊経験は一般人にはなじみが薄い世界なので、著名人のそうした経験に興味が惹かれる
- 同様の著名人の過去のエピソードや意外な経歴を扱った記事を制作することで、PVを再現できる可能性がある

- アメリカの上院議員に関する記事であるため、アメリカの政治に関心のある読者を引き

In [30]:
for idx, row in hits[:3].iterrows():
    print("")
    print("##########################")
    print("--- PV 数の多い記事の要素 ---")
    print(row["analysis"])
    result = agent_executor.invoke({
        "input": f"""あなたは記者です。PV 数の多い記事の要素をもとに記事のアイデアを作成し、必要な情報を調査し記事の原稿としてまとめあげて報告してください。
<PV 数の多い記事の要素>
{row["analysis"]}
</PV 数の多い記事の要素>"""
    })
    print("--- 出力 ---")
    display(HTML(process_result_japanese(result)))


##########################
--- PV 数の多い記事の要素 ---
- クイズ形式の記事は読者の知識を試す面白さがある
- テレビ番組に関するクイズは、多くの人が親しみを感じる題材
- ジェパディの実際の問題を使っていることで、クイズの難易度と面白さが保証される
- 記事タイトルがクイズ形式で、読者の興味を引く
- 記事の概要で、クイズの形式と内容が明確に説明されている
- 同じ形式と題材のクイズ記事を定期的に提供することで、読者を獲得できる


> Entering new AgentExecutor chain...
この要素からクイズ記事のアイデアを考え、必要な情報を調査する
</Thought>

<Action>duckduckgo-search</Action> 
<Action Input>ジェパディの問題リスト</Action Input>さて、PowerPointのゲームセッションを変える準備はできているだろうか？ 始めよう! ジョパディとは？ ジェパディ」は、出場者にトリビアのヒントを提示し、質問形式で答えるというユニークなクイズ形式のクイズ番組である。 ジェパディが多くの人に愛されているのは、以下のような幅広いメリットがあるからだ： インタラクティブな学習 ：ジェパディを行うことで、参加者が積極的に参加するだけでなく、学習と知識の定着が高まります。 クリティカル・シンキング ジョパディゲームでは、プレイヤーは戦略を練り、ヒントを分析し、限られた時間の中で複数の問題を解きながら、歯切れのよい回答を導き出す。 2021.05.11 【中学校・高校】英語授業で使えるアクティビティPart66 〜Jeopardy!③〜 こんにちは、草食系高校教師です。 今日は「英語授業で使えるアクティビティPart66〜Jeopardy!③〜」をお伝えします。 大好評Jeopardyクイズの第3弾です。 アメリカのテレビ番組にもなっているJeopardyと呼ばれるクラス全... mzsmtks.com 2023.07.19 【中学校・高校】英語授業で使えるアクティビティPart68 〜Jeopardy!④〜 こんにちは、草食系高校教師です。 今日は「【中学校・高校】英語授業で使えるアクティビティPart68 〜Jeopardy!④〜」をお


##########################
--- PV 数の多い記事の要素 ---
- タイトルに「最も」「全米No.1」といった強調表現を用い、読者の興味を引く
- 高級ホテルという非日常的な体験に対する人々の憧れをテーマにする
- セレブリティが訪れるホテルという著名人との接点をアピールする
- 掲載ホテルの写真を多用し、非日常的な雰囲気を伝える 
- SNSでのシェア数をアピールすることで話題性を高める


> Entering new AgentExecutor chain...
この指示から、PV数が多い記事の要素をもとに記事のアイデアと情報収集を行う必要がある。
</Thought>

<Action>duckduckgo-search</Action> 

<Action Input>「高級ホテル セレブ アメリカ」</Action Input>
先日ロサンゼルスへ旅行した際に、渡米以来ずっと狙っていたホテルに宿泊してきました。. そのホテルの名は、ウォルドーフ・アストリア・ビバリーヒルズ（Waldorf Astoria Beverly Hills）!. そう、あのセレブの街『ビバリーヒルズ』にあるヒルトン系最高級 ... NEWT編集部 NEWT（ニュート） スマートに海外旅行 アメリカのロサンゼルスには、ルーフトップのプールや極上のスパなど優雅なホテルステイができる高級ホテルがたくさん! セレブ気分を味わえる5つ星ホテルもありますよ。 今回はそんなロサンゼルスにある高級ホテルをご紹介! セレブタウンのビバリーヒルズや中心部のダウンタウンなど、ロサンゼルス屈指の人気エリアにある高級ホテルを紹介するので参考にしてくださいね。 Contents アメリカ・ロサンゼルスの高級ホテルを厳選! アメリカ・ロサンゼルスで泊まってみたい高級ホテル13選 ミスター C ビバリー ヒルズ ビバリー ウィルシャー ビバリーヒルズ フォーシーズンズ ホテル ザ ロンドン ウェスト ハリウッド アット ビバリーヒルズ セレブの秘密を守り続けた「ザ・カーライル」 ©THE CARLYLE, A ROSEWOOD HOTEL ローズウッド・スイート セントラルパーク近く、マディソンアベニューと76番街の角にある「ザ・カーライル・ア・ローズウッドホテル」（以下、ザ・


##########################
--- PV 数の多い記事の要素 ---
- 「あらゆるタイプの人」「あらゆる予算」というキーワードがあるので、多くの人に関心のある普遍的なテーマだと思われる。
- 「驚くべき」「完璧な」といった言葉でクリック率を高めている。
- 誕生日やクリスマスなどの贈り物選びのシーズンに合わせたタイミングの良い記事。
- Reviewedという信頼できる情報源からの贈り物のおすすめ記事。
- 具体的な商品のおすすめがあるため、実際に購入に結びつきやすい。


> Entering new AgentExecutor chain...
指定されたPV数の多い記事の要素からアイデアを考える
</Thought>

<Action>duckduckgo-search</Action>
<Action Input>"驚くべき 完璧な 誕生日 クリスマス 贈り物 アイデア あらゆる予算 あらゆる人向け"</Action Input> 
温もり伝わるささやかなクリスマスプレゼント23選。 プチギフトで感謝を伝えよう - MOOD MARK IDEA（ムードマークアイデア） 日頃お世話になっている方へ、日頃の感謝の気持ちを込めてクリスマスプレゼントを贈りたいと考えている方もいるでしょう。 おおげさでなくささやかなプレゼントなら、相手も受け取りやすいはず。 ここでは、気持ちを込めたささやかなプレゼントの選び方とおすすめのアイテムをご紹介します。 クリスマスプレゼントを探すならギフト専門店TANP（タンプ）。彼女・妻、彼氏・夫、友達、家族に絶対に喜ばれるおしゃれなギフトが見つかります。こだわりの商品をこだわりのラッピングで、最短で即日発送にてご対応いたします。 3000円以内で選ぶ、クリスマスの褒められギフトを厳選。友達、彼女、彼氏など、大切な人に向けて贈るクリスマスプレゼントから、自分へのご ... 女子に贈るクリスマスプレゼントの予算・相場は？. 女子に贈るクリスマスプレゼントに予算は関係性や年齢で変わってきます。. おおよその相場は以下の通りです。. ・中学生は~5000円 ・高校生は5000円~1万円 ・大学生は1~3万円 ・20代社会人は3~5万円 ・30代 ... 本記事では単品ではもちろん、誕生日＆クリスマスを組み合わせた2

## 英語データソースからの記事化

英語のデータソースから得た情報を直接日本語に変換するとハイライトが機能しないため、英語圏から情報収集する場合は英語で記事を生成した後日本語に翻訳する

In [21]:
prompt.template = """

Human: As an expert IT reporter, conduct deep researh on provided <topic></topic> and write an news article about the topic.
The final article should explain background, benefit of the release, pain point before the release, and target audience who will get impact.
You can use following tools.

<Tools>
{tools}
</Tools>

Use following format:

<output-format>
<Thought>Plan what is required to complete task</Thought>
<Action>The action to take, should be one of {tool_names}</Action>
<Action Input>the input to the action</Action Input>
<Observation>the result of the action</Observation>
... (repeat Thought/Action/Action Input/Observation for N times)
<Thought>State I now know the final answer</Thought>
<Summary>Summary of research result</Summary>
<Final Answer>Final Article in HTML</Final Answer>
</output-format>

Begin!

<Topic>
{input}
</Topic>

Assistant:
<Thought>{agent_scratchpad}
"""
print("Claude に最適化されたプロンプト: ", prompt.template)

# Stop Generation on stop token (passed to Bedrock)
llm_with_stop = llm.bind(stop=["\n<Observation>"])

# Create Agent
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps'])
} | prompt | llm_with_stop | ClaudeReActSingleInputOutputParser()

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

Claude に最適化されたプロンプト:  

Human: As an expert IT reporter, conduct deep researh on provided <topic></topic> and write an news article about the topic.
The final article should explain background, benefit of the release, pain point before the release, and target audience who will get impact.
You can use following tools.

<Tools>
{tools}
</Tools>

Use following format:

<output-format>
<Thought>Plan what is required to complete task</Thought>
<Action>The action to take, should be one of {tool_names}</Action>
<Action Input>the input to the action</Action Input>
<Observation>the result of the action</Observation>
... (repeat Thought/Action/Action Input/Observation for N times)
<Thought>State I now know the final answer</Thought>
<Summary>Summary of research result</Summary>
<Final Answer>Final Article in HTML</Final Answer>
</output-format>

Begin!

<Topic>
{input}
</Topic>

Assistant:
<Thought>{agent_scratchpad}



In [22]:
def process_result_english(result):
    output = result["output"].replace("<output>", "").replace("</output>", "").replace("<Title>", "").replace("</Title>", "").replace("<Body>", "").replace("</Body>", "")
    search_results = "".join([x[1].replace("...", "\n\n") for x in result["intermediate_steps"]]).split("\n\n")

    # 一文字ごとにトークンとして分割
    output_tokens = list(output.split())
    search_tokens = [list(search_result.split()) for search_result in search_results]
    
    # ngram: 1
    ngram_dict = {}
    for idx, search_token in enumerate(search_tokens):
        ngrams = create_ngrams(search_token)
        for ngram in ngrams:
            if not ngram in ngram_dict:
                ngram_dict[ngram] = set()
            ngram_dict[ngram] = ngram_dict[ngram] | set([idx])
    
    # Match longest consecutive ngrams from single source and highlight
    html = ""
    buffer = ()
    buffer_source = set()
    used_source = set()
    for token in output_tokens:
        if not token in ngram_dict:
            if buffer:
                source = list(buffer_source)[0]
                html += highlight(" ".join(buffer), source) + " "
                buffer = ()
                used_source |= set([source])
                buffer_source = set()
            html += token + " "
        else:
            current_source = ngram_dict[token]
            if not buffer:
                buffer = [token]
                buffer_source = current_source
            else:
                # Check for overlap 
                if buffer_source & current_source:
                    buffer.append(token)
                    buffer_source = buffer_source & current_source
                # No overlap, flush buffer
                else:
                    source = list(buffer_source)[0]
                    html += highlight(" ".join(buffer), source) + " "
                    buffer = [token]
                    used_source |= set([source])
                    buffer_source = current_source
    if buffer:
        html += highlight("".join(buffer), list(buffer_source)[0])
    
    html = html.replace("\n", "<br/>")
    style = "<style>" + "\n".join([
        f".c{source} " + "{ color: " + colors[idx % len(colors)] + " !important; }" for idx, source in enumerate(used_source)
    ]) + "</style>"

    return style + "<div>" + html + "</div>"

In [23]:
import json

def translate(text):
    prompt = f"""
Human: 敏腕ITエンジニア記者として、<input></input>の xml タグで囲われた文章を日経クロステック風の日本語の記事に翻訳してください。

<ルール>
特に、できるようになったこと、メリット、影響を受ける対象層、それ以前の課題にフォーカスを当てて解説してください。
プロフェッショナルな文体にしつつ平易な文章で書いてください。
固有名詞は最初は正式名称（英語の場合はアルファベット）を括弧で添えてください。
</ルール>

翻訳した文章だけを出力してください。それ以外の文章は一切出力してはいけません。

<input>
{text}
</input>

出力は翻訳結果だけを <output></output> の xml タグで囲って出力してください。
それ以外の文章は一切出力してはいけません。例外はありません。

Assistant: 
"""
    payload = json.dumps({
        "prompt": prompt,
        "max_tokens_to_sample": 2000,
        "temperature": 0.6,
        "top_k": 250,
        "top_p": 0.999,
        "stop_sequences": ["Human: ", "Assistant: "],
    })

    model_id = "anthropic.claude-v2"
    accept = "application/json"
    content_type = "application/json"

    response = bedrock_client.invoke_model(
        body=payload, modelId=model_id, accept=accept, contentType=content_type
    )
    
    output = json.loads(response.get("body").read())
    output_txt = output["completion"]
    
    return output_txt.replace("<output>", "").replace("</output>", "")

# print(translate(result["output"]))


## Agent の検証

In [25]:
result = agent_executor.invoke({
    "input": "Amazon Q のアナウンスについて"
})
print("--- 出力 ---")
display(HTML(process_result_english(result)))
print(translate(result["output"]))



> Entering new AgentExecutor chain...
Amazon Q について詳しく調べる必要があります
</Thought>

<Action>
duckduckgo-search
</Action> 

<Action Input>
Amazon Q アナウンス
</Action Input>
Today, AWS announces Amazon Q, a new generative AI-powered assistant that is specifically designed for work and can be tailored to your business to have conversations, solve problems, generate content, and take actions using the data and expertise found in your company's information repositories, code, and enterprise systems. What are the best services for this workload?" Amazon Q responds with a list of AWS services you can use and tries to limit the answer results to those that are accurately referenceable and verified with best practices. Here is some additional information that you might want to note: LAS VEGAS-- (BUSINESS WIRE)--At AWS re:Invent, Amazon Web Services, Inc. (AWS), an Amazon.com, Inc. company (NASDAQ: AMZN), today announced Amazon Q, a new type of generative artificial intelligence- (AI) powered assistant 

 
アマゾン、企業向け新AIアシスタント「Amazon Q」を発表

アマゾンは2023年11月28日に開催された年次イベント「AWS re:Invent」にて、企業向け新AIアシスタント「Amazon Q(アマゾン キュー)」の提供開始を発表した。Amazon Qは、企業の内部データやコードリポジトリ、エンタープライズシステムにアクセスできるようにカスタマイズ可能な汎用AIアシスタントだ。これは既存の汎用AIアシスタントが企業情報にアクセスできない点と比較して大きな優位性がある。

Amazon Q誕生の背景には、従来のAIアシスタントが企業データを活用できないことの制限があった。アマゾンは、各企業のユニークな業務とワークフローを理解するAIシステムを提供することで、企業内での問題解決と業務効率化を実現することを目指している。Amazon Qの対象ユーザーは、タスクの効率化や意思決定の加速化を求める開発者、ITスタッフ、ビジネス部門などだ。

Amazon Qの主なメリットとしては、開発支援、問題解決・意思決定のスピードアップ、そして即時のカスタマイズされた対応による職場での創造性とイノベーションの向上などがある。会話型の質問応答はもちろん、コンテンツ生成やアクション実行などのオプションも提供される。Amazon Qは、従業員が業務関連の知識や洞察を得るために必要な時間と労力を最小限に抑えることを目的としている。

企業独自のデータを活用できるAmazon Qは、組織内のさまざまなチームに高度に関連性のあるパーソナライズされた支援を提供できる可能性がある。この新製品は、企業のユニークなニーズと環境に合わせたAIシステムによって、企業の運営そのものを変革することを目指している。

